<a href="https://colab.research.google.com/github/sira-p/marmo-4b/blob/main/Marmo_4b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part One: Transfer Learning from Pre-trained Pose Detection Model

In [ ]:
#download training dataset
!rm -rf sample_data
!test -d train || (curl -o train.zip https://zenodo.org/records/5849371/files/marmoset-dlc-2021-05-07.zip?download=1 && unzip -q train.zip -d ./train && rm -f train.zip)

In [ ]:
#install ultralytics and deeplabcut2yolo packages via pip
!pip install ultralytics deeplabcut2yolo

In [ ]:
#obsolete
def mount_drive():
  import locale
  def getpreferredencoding(do_setlocale = True):
      return "UTF-8"
  locale.getpreferredencoding = getpreferredencoding

  from google.colab import drive
  drive.mount('/content/drive')

mount_drive()

In [ ]:
#load deeplabcut packages and specify input-output path
import deeplabcut2yolo as d2y

json_path = "./train/marmoset-dlc-2021-05-07/training-datasets/iteration-0/UnaugmentedDataSet_marmosetMay7/dlc_shuffle1_train.json"
csv_path = "./train/marmoset-dlc-2021-05-07/training-datasets/iteration-0/UnaugmentedDataSet_marmosetMay7/CollectedData_dlc.csv"
root_dir = "./train/marmoset-dlc-2021-05-07/labeled-data/"

#initiate conversion from deeplabcut(coco-like structure to yolov8 format)
d2y.convert(json_path, csv_path, root_dir, datapoint_classes=[0, 1], n_keypoint_per_datapoint=30, precision=6)

In [ ]:
#define function for create configuration file (.yaml)
def create_data_yml(train_path, val_path, kpt_shape, flip_idx, nc, names, output_path):
  data = {
    "train": train_path,
    "val": val_path,
    "kpt_shape": kpt_shape,
    "flip_idx": flip_idx,
    "nc": nc,
    "names": names
  }

  with open(output_path, "w") as f:
    import yaml
    yaml.dump(data, f, default_flow_style=None, sort_keys=False)

In [ ]:
#defining variables for file path and configuration for configuration file (.yaml)
train_path = "/content/train/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1"
val_path = "/content/train/marmoset-dlc-2021-05-07/labeled-data/refinement1"
data_path = "/content/data.yml"
shape = [15, 3]
flip_index = [0, 3, 2, 1, 6, 7, 4, 5, 9, 8, 11, 10, 12, 13, 14]
nc = 2
names = ["B", "W"]

#loading function create_data_yml to create configuration file (.yaml)
create_data_yml(train_path, val_path, shape, flip_index, nc, names, data_path)

In [ ]:
#loading YOLOv8 packages
from ultralytics import YOLO

#load YOLOv8 pose estimation pretrained model
model = YOLO("yolov8n-pose.pt").load('yolov8n-pose.pt')
#fine tune the pretrained model with the marmoset dataset from deeplabcut
model.train(data="data.yml", epochs=50, imgsz=640)

In [ ]:
#move files to a new directory
!mv /content/runs/pose/train /content/drive/MyDrive/marmo-4b/

Optionally, train a one-class pose detection model.

In [ ]:
#load deeplabcut packages and specify input-output path
import deeplabcut2yolo as d2y

json_path = "./train/marmoset-dlc-2021-05-07/training-datasets/iteration-0/UnaugmentedDataSet_marmosetMay7/dlc_shuffle1_train.json"
csv_path = "./train/marmoset-dlc-2021-05-07/training-datasets/iteration-0/UnaugmentedDataSet_marmosetMay7/CollectedData_dlc.csv"
root_dir = "./train/marmoset-dlc-2021-05-07/labeled-data/"

#initiate conversion from deeplabcut(coco-like structure to yolov8 format)
d2y.convert(json_path, csv_path, root_dir, datapoint_classes=[0, 0], n_keypoint_per_datapoint=30, precision=6)

In [ ]:
#defining variables for file path and configuration for configuration file (.yaml)
train_path = "/content/train/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1"
val_path = "/content/train/marmoset-dlc-2021-05-07/labeled-data/refinement1"
data_path = "/content/data.yml"
shape = [15, 3]
flip_index = [0, 3, 2, 1, 6, 7, 4, 5, 9, 8, 11, 10, 12, 13, 14]

#initiate conversion from deeplabcut(coco-like structure to yolov8 format) by specifying class as 1, the model is trained only with one marmoset
create_data_yml(train_path, val_path, shape, flip_index, 1, ["marmoset"], data_path)

In [ ]:
#loading YOLOv8 packages
from ultralytics import YOLO

#load YOLOv8 pose estimation pretrained model
model_one_class = YOLO("yolov8n-pose.pt").load('yolov8n-pose.pt')
#fine tune the pretrained model with the marmoset dataset from deeplabcut
model_one_class.train(data="data.yml", epochs=50, imgsz=640)

In [ ]:
#move trained files to a new directory
!mv /content/runs/pose/train /content/drive/MyDrive/marmo-4b-one/

# Part Two: SAM Mask Generation Using Points from Pose Detection

In [2]:
#workaround for notebook git clone bug
import locale
locale.getpreferredencoding = lambda: "UTF-8"
#download pretrained pose detection model (from part 1) from github
!git clone https://github.com/sira-p/marmo-4b.git
!rm -rf sample_data
#download segmentanything from github
!find sam_vit_h.pth || curl -o sam_vit_h.pth https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
#download the test dataset
!test -d test || (curl -o test.zip https://zenodo.org/records/8437121/files/marmoset-dlc-2021-05-07.zip?download=1 && unzip -q test.zip -d ./test && rm -f test.zip)
#install segmentanything and ultralytics from pip
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install ultralytics

Cloning into 'marmo-4b'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 38 (delta 12), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (38/38), 17.59 MiB | 15.55 MiB/s, done.
Resolving deltas: 100% (12/12), done.
sam_vit_h.pth
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-ltor72w1
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-ltor72w1
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 526fd066dea338ba2ca08886853bd37ffd6a8aec
  Preparing metadata (setup.py) ... done


In [3]:
#loading dependecies
from ultralytics import YOLO
import glob
from segment_anything import SamPredictor, sam_model_registry
import numpy as np
import torch
import cv2
import os
from tqdm import tqdm

In [4]:
#define the Marmo-4b model class
class Marmo4b:
  #defining the color for the masks and skeletons connections
  COLORS = [
      (255, 0, 0),
      (0, 255, 0),
      (0, 0, 255),
      (255, 255, 0),
      (255, 0, 255),
      (0, 255, 255),
    ]
  SKELETON = [
    (0, 1),
    (0, 2),
    (0, 3),
    (3, 7),
    (7, 8),
    (5, 12),
    (13, 14),
    (11, 14),
    (3, 9),
    (9, 10),
    (3, 4),
    (4, 5),
    (6, 13),
    (6, 11),
  ]
  CONFIDENCE_THRESHOLD = 0.5

#defining initiation function for SAM
  def __init__(self, pose_model_path, sam_checkpoint_path):
    self.pose_model = YOLO(pose_model_path)
    sam = sam_model_registry["default"](checkpoint=sam_checkpoint_path)
    sam = sam.to(device='cuda' if torch.cuda.is_available() else 'cpu')
    self.mask_predictor = SamPredictor(sam)

#defining function for color masks
  @staticmethod
  def apply_color_masks(image, masks):
    for i, mask in enumerate(masks):
      mask = cv2.cvtColor((mask[0].squeeze() * 255).astype('uint8'), cv2.COLOR_GRAY2BGR)
      color_mask = np.zeros_like(image)
      color_mask[:,:] = Marmo4b.COLORS[i]
      color_mask[mask == 0] = 0

      image = cv2.addWeighted(image, 0.7, color_mask, 0.3, 0)
    return image

#defining function for joint points
  @staticmethod
  def draw_joint(image, points):
    for i in range(len(points)):
      for x, y in points[i]:
        image = cv2.circle(image, (int(x), int(y)), 2, Marmo4b.COLORS[i], -1)
    return image

#defining function to draw skeleton connections
  @staticmethod
  def draw_skeleton(image, points, masks):
    overlay = image.copy()
    for i in range(len(points)):
      mask = masks[i]
      point = points[i]
      for start_i, end_i in Marmo4b.SKELETON:
        if mask[start_i] and mask[end_i]:
          # If confident, then draw
          overlay = cv2.line(overlay, tuple(point[start_i].astype(int)), tuple(point[end_i].astype(int)), Marmo4b.COLORS[i], 2)

    image = cv2.addWeighted(image, 0.3, overlay, 0.7, 0)

    return image

#defining funtion for drawing bounding boxes
  @staticmethod
  def is_within_box(bboxes, points):
    mask = np.zeros((points.shape[0], points.shape[1]), dtype=bool)
    for bbox in bboxes:
      in_x = (points[:, :, 0] >= bbox[0]) & (points[:, :, 0] <= bbox[2])
      in_y = (points[:, :, 1] >= bbox[1]) & (points[:, :, 1] <= bbox[3])

      mask = mask | (in_x & in_y)
    return mask

#defining function for using both models together
  def predict_image(self, image, mode="points"):
    """
    Args:
      image (OpenCV image object)
    """
    if image is None:
      raise ValueError("Image not provided")

    MODES = ["points", "box"]
    if not mode in MODES:
      raise ValueError(f"Invalid mode of operation: use {' '.join(MODES)}")

    pose_result = self.pose_model(image)[0]

    joint_points = pose_result.keypoints.xy.cpu().numpy()
    boxes = pose_result.boxes.xyxy.cpu().numpy()
    point_confidences = pose_result.keypoints.conf.cpu().numpy()
    # box_mask = is_within_box(boxes, joint_points)
    confidence_mask = point_confidences > self.CONFIDENCE_THRESHOLD
    zero_mask = ~np.all(joint_points[:, :, :] == np.array([0, 0]), axis=2)
    point_mask = zero_mask & confidence_mask
    input_points = np.where(np.expand_dims(point_mask, axis=2), joint_points, 0)

    self.mask_predictor.set_image(image)

    if mode == "points":
      input_labels = point_mask.astype(int)
      sam_masks = [self.mask_predictor.predict(point_coords=point, point_labels=label, multimask_output=False) for point, label in zip(input_points, input_labels)]

    if mode == "box":
      sam_masks = [self.mask_predictor.predict(box=box, multimask_output=False) for box in boxes]

    image = self.apply_color_masks(image, sam_masks)
    image = self.draw_joint(image, input_points)
    image = self.draw_skeleton(image, joint_points, point_mask)

    return image

  def predict_images(self, image_paths, save_path, mode="points"):
    os.makedirs(save_path, exist_ok=True)

    for image_path in tqdm(image_paths):
      image = cv2.imread(image_path)
      result = self.predict_image(image, mode)
      cv2.imwrite(save_path + image_path.split("/")[-1], result)

In [11]:
#initializing the model
model = Marmo4b("marmo-4b/marmo-pose-one-class.pt", "/content/sam_vit_h.pth")

In [ ]:
#single image demo
image = cv2.imread("/content/test/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1/img01310.png")
result = model.predict_image(image, mode="box")
cv2.imwrite("processed.png", result)

In [ ]:
#setting image path for prediction
image_paths = glob.glob("/content/test/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1/*.png")
#batch prediction
model.predict_images(image_paths, "/content/predict/", mode="box")

In [6]:
#image to video conversion function
def images_to_video(image_directory, video_name, fps=30, target_size=(500, 500)):

  image_paths = glob.glob(image_directory + "*.png")
  image_paths.sort()

  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  video = cv2.VideoWriter(video_name, fourcc, fps, target_size)

  for image_path in image_paths:
    frame = cv2.imread(image_path)
    resized_frame = cv2.resize(frame, target_size)
    video.write(resized_frame)

  video.release()

In [ ]:
#convert image to video
images_to_video("/content/predict/", "myvideo.mp4")

# Part Three: Implementing Interface

In [12]:
#installing Gradio via pip
!pip install gradio

In [13]:
#loading prerequisites
import gradio as gr
from PIL import Image
import os

In [14]:
#defining function for batch prediction (multiple images)
def batch_predict(image_paths, mode, model):
  temp_dir = "temp_results/"
  os.makedirs(temp_dir, exist_ok=True)

  model.predict_images(image_paths, temp_dir, mode)
  result_paths = glob.glob(temp_dir + "*")

  return [Image.open(result_path) for result_path in result_paths]

#defining gradio's interface
def launch_interface(model):
  gr.Interface(
      fn=lambda image_list, mode: batch_predict(image_list, mode, model),
      inputs=[
          gr.File(label="Upload images", file_count="multiple", type="filepath"),
          gr.Radio(choices=["points", "box"], label="Segmentation mode"),
      ],
      outputs=gr.Gallery(label="Processed Images"),
      title="Pose Estimation and Instance Segmentation",
      description="Upload images and select the mode (points or box) to predict poses and masks."
  ).launch(debug=True)

In [ ]:
#launch gradio webapp
launch_interface(model)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://93e53a20a7a982a014.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


  0%|          | 0/1 [00:00<?, ?it/s]


0: 640x640 1 marmoset, 9.8ms
Speed: 3.2ms preprocess, 9.8ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)


  0%|          | 0/1 [00:00<?, ?it/s]


0: 640x640 1 marmoset, 8.4ms
Speed: 2.7ms preprocess, 8.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


100%|██████████| 1/1 [00:02<00:00,  2.31s/it]
